# LH114-7

Updating the dataset: D098110100000

* Only the LiF 1 and SiC 1 channels have flux.  
* We scale SiC 1A to match LiF 1A. 
* SiC 1B and LiF 1A overlap, but the continuum shapes differ enough that we can't compute a scale factor. Instead, we adopt the value computed for SiC 1A.
* LiF 1B looks faint compared to LiF 1A, but the count-rate is solid and the exposure time differs from that of LiF 1A by only 4%.  
* We conclude that the LiF 1B flux is OK.  
* We scale to match the STIS spectrum at long wavelengths. 
* The final spectrum has a couple of gaps, but looks pretty good.

COMMENT File updated 21 September 2022<br>
COMMENT This NVO file has been reconstructed from the corresponding ALL file.<br>
COMMENT Only the LiF 1 and SiC 1 channels are available. <br>
COMMENT Spectrum scaled by 1.31 to match STIS.

In [ ]:
%matplotlib inline
import matplotlib

import numpy as np

from astropy import units as u
from astropy.convolution import convolve, Box1DKernel
from astropy.io import fits
from astropy.visualization import quantity_support
from matplotlib import pyplot as plt

from specutils import Spectrum1D
from specutils.spectra import SpectralRegion
from specutils.manipulation import box_smooth, extract_region

# Specify plot parameters.
quantity_support()  # to put units on the axes below 
matplotlib.rcParams['figure.figsize'] = [15, 5]

# Wavelength per channel (pixel) is always 0.013 A.
WPC = 0.013

# Normalize all spectra, so fluxes are of order unity.
norm = 1E13

In [ ]:
# Function to convert FITS array into spectrum object.

def make_spectrum (specdata):
    lamb = specdata['wave'] * u.AA 
    flux = specdata['flux'] * norm * u.Unit('erg cm-2 s-1 AA-1') 
    return Spectrum1D(spectral_axis=lamb, flux=flux)

In [ ]:
# Function to compute offset in pixels between two spectra

def compute_shift(specdata, refdata, wmin, wmax):
    wave_spec  = specdata['wave']
    flux_spec  = specdata['flux']
    error_spec = specdata['error']
    wave_ref   =  refdata['wave']
    flux_ref   =  refdata['flux']
    
    # Compute scale factor between data and reference spectra.
    g = np.where((wave_spec > wmin) & (wave_spec < wmax))
    h = np.where((wave_ref  > wmin) & (wave_ref  < wmax))
    spec_mean = np.mean(flux_spec[g])
    ref_mean  = np.mean(flux_ref[h])
    scale = ref_mean / spec_mean

    # Smooth data and error arrays.
    flux_spec  = convolve(flux_spec,  Box1DKernel(7))
    error_spec = convolve(error_spec, Box1DKernel(7))
    flux_ref   = convolve(flux_ref,   Box1DKernel(7))
    
    # Rescale spectrum to match reference.
    flux_spec *= scale
    error_spec *= scale
    
    # Compute minimum value of chi-squared.
    chisq = np.zeros(20)
    for i in range(20):
        j = i - 10
        chisq[i] = np.sum(((flux_spec[g[0]+j] - flux_ref[h]) / error_spec[g[0]+j])**2)
    
    # We have computed this offset by shifting the flux array, but will 
    # use it by shifting the wavelength array, so must multiply by -1.
    return 10 - np.argmin(chisq)

In [ ]:
# Read header keywords.

filename = 'd098110100000all2ttagfcal.fit'
f = fits.open(filename)
print ('Target:  ', f[0].header['TARGNAME'])
print ('Aperture:', f[0].header['APERTURE'])
print ('Guider:  ', f[0].header['FESCENT'])
print ('CHANL OBSTIME COMBMETH')
for i in range(1,9): print (f[i].header['EXTNAME'], f[i].header['OBSTIME'], f[i].header['COMBMETH'])
    
# The guide channel was LiF 1.

In [ ]:
# Read all eight spectra from FUSE *all* file.

lif1a_data = f['1alif'].data 
lif1b_data = f['1blif'].data
lif2b_data = f['2blif'].data
lif2a_data = f['2alif'].data
sic1a_data = f['1asic'].data
sic1b_data = f['1bsic'].data
sic2b_data = f['2bsic'].data
sic2a_data = f['2asic'].data
f.close() 

In [ ]:
# Convert FITS arrays into spectrum objects.

lif1a = make_spectrum (lif1a_data)
lif1b = make_spectrum (lif1b_data)
lif2a = make_spectrum (lif2a_data)
lif2b = make_spectrum (lif2b_data)
sic1a = make_spectrum (sic1a_data)
sic1b = make_spectrum (sic1b_data)
sic2a = make_spectrum (sic2a_data)
sic2b = make_spectrum (sic2b_data)

In [ ]:
# Smooth the spectral arrays.

from specutils.manipulation import (box_smooth)

lif1a_bsmooth = box_smooth(lif1a, width=15)
lif1b_bsmooth = box_smooth(lif1b, width=15)
lif2a_bsmooth = box_smooth(lif2a, width=15)
lif2b_bsmooth = box_smooth(lif2b, width=15)
sic1a_bsmooth = box_smooth(sic1a, width=15)
sic1b_bsmooth = box_smooth(sic1b, width=15)
sic2a_bsmooth = box_smooth(sic2a, width=15)
sic2b_bsmooth = box_smooth(sic2b, width=15)

In [ ]:
# Plot the smoothed combined spectra.  
# Note that spectra are scaled to have values near unity.

f, ax = plt.subplots()  
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(lif2b_bsmooth.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B') 
ax.step(sic1a_bsmooth.spectral_axis, sic1a_bsmooth.flux, label='SiC 1A') 
ax.step(sic2b_bsmooth.spectral_axis, sic2b_bsmooth.flux, label='SiC 2B') 
ax.legend()
ax.set_ylim([0,20])

# Only the LiF 1 and SiC 1 channels were pointed at the star.  We ignore the others. 

In [ ]:
# Rescale the SiC 1A channel to match LiF 1A.

# Select a broad spectral region.

region = SpectralRegion(1040*u.AA, 1070*u.AA)
sub_lif1a = extract_region(lif1a, region)
sub_sic1a = extract_region(sic1a, region)

# Compute ratio of their fluxes.

scale_sic1a = sub_lif1a.mean()/sub_sic1a.mean()
print ('Scale SiC 1A by', scale_sic1a)

In [ ]:
# Scale the smoothed spectrum.

sic1a_bsmooth *= scale_sic1a

In [ ]:
# Replot the rescaled spectra.

f, ax = plt.subplots()  
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(sic1a_bsmooth.spectral_axis, sic1a_bsmooth.flux, label='SiC 1A')
ax.legend()
ax.set_ylim([0,20])

# Generally good, but the SiC 1A spectrum is a bit brigher beyond 1070 A.

In [ ]:
# Let's see what's happening at long wavelengths.

f, ax = plt.subplots()  
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(lif1b_bsmooth.spectral_axis, lif1b_bsmooth.flux, label='LiF 1B') 
ax.legend()
ax.set_ylim([0,20])

# Compared to LiF 1A, the LiF 1B flux looks a little low.  But the count-rate plots show that
# the LiF 1B count rate is steady, and its exposure time differs from that of LiF 1A by only 4%.
# (Remember that LiF 1A and LiF 1B were taken with the same spectrograph, but different detectors.)
# We conclude that, relative to LiF 1A, the LiF 1B spectrum is properly scaled.  Any further
# adjustments (to match the STIS spectrum, below) apply to the entire spectrum.

In [ ]:
# Compare with a STIS spectrum of the same star.

filename = 'oeb72s020_x1d.fits'
print ('Target:', fits.getval(filename, 'TARGNAME'))
print ('Aperture:', fits.getval(filename, 'APERTURE'))
stis = Spectrum1D.read(filename, format="HST/STIS")
stis_bsmooth = box_smooth(stis, width=15)

f, ax = plt.subplots()
ax.plot(stis_bsmooth.spectral_axis, norm * stis_bsmooth.flux, label='STIS', color='g')  
ax.plot(lif1b_bsmooth.spectral_axis, lif1b_bsmooth.flux, label='LiF 1B')
ax.legend()
ax.set_xlim([1140,1300])
ax.set_ylim([0,20])

# The FUSE spectrum is too faint.  We'll compute a scale factor for the entire spectrum.

In [ ]:
# Select a spectral region.

region = SpectralRegion(1155*u.AA, 1165*u.AA)
sub_lif1b = extract_region(lif1b, region)
sub_stis = extract_region(stis, region)

# Compute ratio of their fluxes.

mean = sub_stis.mean()
scale_entire_spectrum = norm * mean/sub_lif1b.mean()
print ('Scale spectrum by', scale_entire_spectrum)

In [ ]:
# Plot the rescaled spectra.

f, ax = plt.subplots()  
ax.plot(stis_bsmooth.spectral_axis, norm * stis_bsmooth.flux, label='STIS')  
ax.step(lif1b_bsmooth.spectral_axis, scale_entire_spectrum * lif1b_bsmooth.flux, label='LiF 1B') 
ax.legend()
ax.set_xlim([1090, 1240])
ax.set_ylim([0,20])

# This looks OK, but the absorption lines don't match perfectly.

In [ ]:
# Let's see whether the scale factor for SiC 1A works for SiC 1B.

f, ax = plt.subplots() 
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(sic1b_bsmooth.spectral_axis, scale_sic1a * sic1b_bsmooth.flux, label='SiC 1B')
ax.legend()
ax.set_xlim([920, 1020])
ax.set_ylim([0,20])

# We can't do better than this, because the region of overlap is a mess for SiC 1B.

In [ ]:
# Next we compute the shift (in pixels) relative to LiF 1A.

shift_sic1 = compute_shift(sic1a_data, lif1a_data, 1040, 1070)

print ('Shift SiC 1 by ', shift_sic1, ' pixels.')

In [ ]:
# Shift the SiC 1 spectrum.

shift_sic1 *= WPC

lif1a_shift = lif1a
lif1b_shift = lif1b
sic1a_shift = Spectrum1D(spectral_axis=sic1a.spectral_axis + shift_sic1 * u.AA, flux=sic1a.flux)
sic1b_shift = Spectrum1D(spectral_axis=sic1b.spectral_axis + shift_sic1 * u.AA, flux=sic1b.flux)

f, ax = plt.subplots()  
ax.step(lif1a.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(sic1a_shift.spectral_axis, sic1a_bsmooth.flux, label='SiC 1A') 
ax.legend()
ax.set_xlim([1060,1070])
ax.set_ylim([0,20])

In [ ]:
# We'll do the best we can with the available spectral segments.
# I'm going to leave a gap between the SiC 1B and LiF 1A spectra, 
# because otherwise it will look like a broad absorption line.

filename = 'd098110100000nvo2ttagfcal.fit'
f = fits.open(filename)
hdr = f[0].header
data = f[1].data 

# Start by setting everything to zero.
data['flux'] = 0.
data['error'] = 0.

# Scale SiC 1B
sic1b_wave = sic1b_shift.spectral_axis.value
h = np.where((sic1b_wave > 899.99) & (sic1b_wave < 985))
g = np.where((data['wave'] > np.min(sic1b_wave[h]) - 0.01) & (data['wave'] < 985))
data['flux'][g] = float(scale_sic1a) * sic1b_data['flux'][h]
data['error'][g] = float(scale_sic1a) * sic1b_data['error'][h]

# Scale LiF 1A
lif1a_wave = lif1a_shift.spectral_axis.value
g = np.where((data['wave'] > 988) & (data['wave'] < 1082.5))
h = np.where((lif1a_wave > 988) & (lif1a_wave < 1082.5))
data['flux'][g] = lif1a_data['flux'][h]
data['error'][g] = lif1a_data['error'][h]

# Scale SiC 1A 
sic1a_wave = sic1a_shift.spectral_axis.value
g = np.where((data['wave'] > 1082.5) & (data['wave'] < 1092))
h = np.where((sic1a_wave > 1082.5) & (sic1a_wave < 1092))
data['flux'][g] = float(scale_sic1a) * sic1a_data['flux'][h]
data['error'][g] = float(scale_sic1a) * sic1a_data['error'][h]

# Scale LiF 1B
lif1b_wave = lif1b_shift.spectral_axis.value
h = np.where((lif1b_wave > 1093) & (lif1b_wave < 1190.01))
g = np.where((data['wave'] > 1093) & (data['wave'] < np.max(lif1b_wave[h]) + 0.01))
data['flux'][g] = lif1b_data['flux'][h]
data['error'][g] = lif1b_data['error'][h]

# Rescale entire spectrum to match STIS.
data['flux'] *= float(scale_entire_spectrum)
data['error'] *= float(scale_entire_spectrum)

hdr['comment'] = ''
hdr['comment'] = 'File updated 21 September 2022'
hdr['comment'] = 'This NVO file has been reconstructed from the corresponding ALL file.'
hdr['comment'] = 'Only the LiF 1 and SiC 1 channels are available.'
hdr['comment'] = 'Spectrum scaled by {0:.2f} to match STIS.'.format(scale_entire_spectrum)

f.writeto('level0_d098110100000nvo2ttagfcal_vo.fit', overwrite=True)
f.close()

In [ ]:
# Compare old and new versions of NVO file.

filename = 'd098110100000nvo2ttagfcal.fit'
f = fits.open(filename)
old = f[1].data 
f.close()

filename = 'level0_d098110100000nvo2ttagfcal_vo.fit'
f = fits.open(filename)
hdr = f[0].header
new = f[1].data 
f.close()

print (hdr['comment'])

f, (ax1, ax2) = plt.subplots(2, 1, sharey=True)  
ax1.step(new['wave'], new['flux'], label='NEW FLUX')
ax1.step(old['wave'], old['flux'], label='OLD FLUX') 
ax2.step(new['wave'], new['flux'], label='NEW FLUX')
ax2.step(old['wave'], old['flux'], label='OLD FLUX') 
ax1.legend()
ax2.legend()
ax1.set_xlim([900, 1050])
ax2.set_xlim([1050, 1190])
ax1.set_ylim([0,2.0E-12])

In [ ]:
# Let's look more closely at the regions sampled only by the SiC channels.

f, (ax1, ax2) = plt.subplots(2, 1, sharey=True)  
ax1.step(new['wave'], new['flux'], label='NEW FLUX')
ax1.step(old['wave'], old['flux'], label='OLD FLUX') 
ax2.step(new['wave'], new['flux'], label='NEW FLUX')
ax2.step(old['wave'], old['flux'], label='OLD FLUX') 
ax1.legend()
ax2.legend()
ax1.set_xlim([980, 1000])
ax2.set_xlim([1075, 1100])
ax1.set_ylim([0,2.0E-12])

# We see again that the SiC 1A spectrum rises towards long wavelengths.